In [1]:
### DATASETs
import datasets
from tokenized_dataset import load_tokenized_dataset_gpt2, load_tokenized_dataset_hellaswag, get_batched_examples, get_batched_examples_packed
ds, (tokenize, detokenize, tokenizer_vocab_size) = load_tokenized_dataset_gpt2("train[:1%]") #:1% or :1000
ds = ds.train_test_split(test_size=0.1, seed=42) # TODO: put seed in better place? does it mess up with resume_from_checkpoint logic?
ds = datasets.DatasetDict({
    'train': ds['train'],
    'validation': ds['test'] #rename
})
print(ds)

hellaswag_ds = load_tokenized_dataset_hellaswag(tokenize)
print(hellaswag_ds)

# Tests:
item = next(x for x in hellaswag_ds)
print(item)
print(detokenize((item['y'],)))
item_x = item['x']
def unpack_hellaswag_x(item_x):
    choices = []
    while 0 in item_x:
        ind=item_x.index(0)
        choices.append(item_x[:ind])
        #print(detokenize((item_x[:ind],))) # choices
        item_x = item_x[ind+1:]
    assert len(item_x)==1
    return choices, item_x[0]
choices, label = unpack_hellaswag_x(item['x'])
print(detokenize(choices)) # TODO XXX: one of chocies has ", while others have '. Is it anything serious?
print(label)

# item_x = item['x']
# while 0 in item_x:
#     ind=item_x.index(0)
#     print(detokenize((item_x[:ind],))) # choices
#     item_x = item_x[ind+1:]
# print(item_x[:ind])

Loading FineWeb-Edu dataset


Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Loading tokenizer bpe_tokenizer_fineweb-edu_sample-10BT_100k_ds_merges_30k.pickle
Tokenizing dataset
DatasetDict({
    train: Dataset({
        features: ['x', 'y'],
        num_rows: 87048
    })
    validation: Dataset({
        features: ['x', 'y'],
        num_rows: 9673
    })
})
Loading HellaSwag dataset
Tokenizing dataset
Dataset({
    features: ['x', 'y'],
    num_rows: 10042
})
{'x': [5419, 6016, 22663, 5409, 22663, 5410, 9759, 5404, 5983, 7069, 0, 5419, 5458, 12663, 6818, 16545, 14281, 0, 5419, 11340, 5410, 6740, 5781, 22926, 6247, 10869, 0, 11375, 20711, 5820, 15975, 5411, 5416, 5410, 34963, 0, 3], 'y': [5410, 6259, 5419, 12716, 5416, 5410, 34963, 5676]}
['a man is sitting on a roof. he ']
['is using wrap to wrap a pair of skis. ', 'is ripping level tiles off. ', "is holding a rubik's cube. ", 'starts pulling up roofing on a roof. ']
3


In [3]:
model_vocab_size = tokenizer_vocab_size + 3 # add padding token (0) + start of sequence token + end of sequence token 
START_TOK = tokenizer_vocab_size + 1
END_TOK = tokenizer_vocab_size + 2 # TODO: in standard LLM convention, it should be 1. Also, it could be part of tokenizer_vocab_size


# Compute HellaSwag score
import numpy as np
def unpack_hellaswag_batched_x(batched_x: np.ndarray):
    choices = []
    labels = []
    for item_x in batched_x:
        print(item_x)
        item_choices, item_label = unpack_hellaswag_x(list(np.trim_zeros(item_x)))
        choices.append(item_choices)
        labels.append(item_label)
    return list(map(list, zip(*choices))), labels


hellaswag_accs = []
for _, batch in enumerate(get_batched_examples(hellaswag_ds, 2, 100, START_TOK, END_TOK, split=None)): 
    batch_losses = []
    x, y, x_mask, y_mask, yx_mask, x_indices, y_indices = batch
    #print(x)
    choices, label = unpack_hellaswag_batched_x(x) #TODO XXX: finish
    for choice in choices:
        #_, (loss_val, acc, toks_prop) = loss_eval(params, jnp.array(x), jnp.array(y), jnp.array(x_mask), jnp.array(y_mask), jnp.array(yx_mask), jnp.array(x_indices), jnp.array(y_indices))
        loss_val = 0
        batch_losses.append(loss_val) # TODO XXX: we need loss per item, or just code up logprob func
    # TODO XXX: select min for each item in batch, then add accs to hellaswag_accs
       
hellaswag_acc = sum(hellaswag_accs)/len(hellaswag_accs)

[ 5419  6016 22663  5409 22663  5410  9759  5404  5983  7069     0  5419
  5458 12663  6818 16545 14281     0  5419 11340  5410  6740  5781 22926
  6247 10869     0 11375 20711  5820 15975  5411  5416  5410 34963     0
     3     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[ 6777  5883  5406  9429  5412  6078 25532     0 29852  5391  6392 14325
 20598     0 29852  5410 23282 11692  5409  5391  6392 11814  5395     0
 11726  5406  5969  5527  5391  7652  5939  6078 14457     0     3     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0 

AssertionError: 